In [1]:
from __future__ import division, print_function
import sys
import os
import glob
import re
import h5py
import numpy as np
from tensorflow.keras.models import Sequential
from keras.models import load_model
from keras.preprocessing import image
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from flask import Flask ,redirect,url_for,request,render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
from werkzeug.serving import run_simple
from werkzeug.wrappers import Request, Response
from PIL import Image

app = Flask(__name__)
model = load_model('my_model (1).h5')
target_img = os.path.join(os.getcwd() , 'static/images')

@app.route('/')
def index_view():
    return render_template('index.html')

#Allow files with extension png, jpg and jpeg
ALLOWED_EXT = set(['jpg' , 'jpeg' , 'png'])
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXT
           
# Function to load and prepare the image in right shape
def read_image(filename):

    img = load_img(filename, target_size=(260, 260))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

@app.route('/predict',methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = file.filename
            file_path = os.path.join('static/images', filename)
            file.save(file_path)
            img = read_image(file_path)
            class_prediction=model.predict(img) 
            classes_x=np.argmax(class_prediction,axis=1)
            if classes_x == 0:
              y = "normal"
            elif classes_x == 1:
              y = "mild"
            elif classes_x == 2:
              y = "moderate"
            elif classes_x == 3:
              y = "severe"
            else:
              y = "proliferativec"
            return render_template('index.html',y=y,prob=class_prediction, user_image = file_path)
        else:
            return "Unable to read the file. Please check file extension"
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False, port=8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jun/2023 21:19:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2023 21:19:34] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 5s 5s/step


127.0.0.1 - - [13/Jun/2023 21:19:55] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2023 21:19:56] "GET /static/images/0a1076183736.png HTTP/1.1" 200 -
